# install and load dependancy

In [1]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00


In [2]:
import os
import sys
import torch
import tqdm
import yaml
import json
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms as T

import logging
from torch.utils.tensorboard import SummaryWriter
from shutil import copyfile

## Link with Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/'
sys.path.append(path)

from dataset import SemEvalDataset, Collate
from models import MemeMultiLabelClassifier
from sampler import MultilabelBalancedRandomSampler

from scorer import evaluate
from format_checker import read_classes

Mounted at /content/drive


# Load data from drive

In [4]:
def load_data(config, path, workers, val_fold):

    # Load data loaders
    test_transforms = T.Compose([T.Resize(256),
                    T.CenterCrop(224),
                    T.ToTensor(),
                    T.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])])
    train_transforms = T.Compose([T.Resize(256),
                    T.RandomCrop(224),
                    T.ToTensor(),
                    T.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])])

    train_dataset = SemEvalDataset(config, path, split='train', transforms=train_transforms, val_fold=val_fold)
    val_dataset = SemEvalDataset(config, path, split='val', transforms=test_transforms, val_fold=val_fold)

    id_intersection = set([x['id'] for x in train_dataset.targets]).intersection([x['id'] for x in val_dataset.targets])
    assert len(id_intersection) == 0

    if config['dataset']['task'] == 3:
        classes = read_classes(path + 'techniques_list_task3.txt')

    collate_fn = Collate(path, config, classes)
    if 'balanced-sampling' in config['training'] and config['training']['balanced-sampling']:
        classes_ids = [[train_dataset.class_list.index(x) for x in info['labels']] for info in train_dataset.targets]
        labels = np.zeros((len(classes_ids), len(train_dataset.class_list)))
        for l, c in zip(labels, classes_ids):
            l[c] = 1
        sampler = MultilabelBalancedRandomSampler(labels)
    else:
        sampler = None

    train_dataloader = DataLoader(train_dataset, batch_size=config['training']['bs'], shuffle=True if sampler is None else False, num_workers=workers, collate_fn=collate_fn, sampler=sampler)
    val_dataloader = DataLoader(val_dataset, batch_size=config['training']['bs'], shuffle=False,
                                  num_workers=workers, collate_fn=collate_fn)

    return train_dataloader, val_dataloader, classes

# Train the model and validate

In [5]:
def train_model(config, path, model, optimizer, scheduler, train_dataloader, val_dataloader, classes, num_epochs, log_step, val_step, val_fold):
    model.train()

    start_epoch = 0
    mean_loss = 0
    progress_bar = tqdm.trange(start_epoch, num_epochs)
    progress_bar.set_description('Train')
    best_f1 = 0.0

    for epoch in progress_bar:
        for it, (image, text, text_len, labels, ids) in enumerate(train_dataloader):
            global_iteration = epoch * len(train_dataloader) + it

            if torch.cuda.is_available():
                image = image.cuda() if image is not None else None
                text = text.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()

            loss = model(image, text, text_len, labels)
            loss.backward()
            optimizer.step()
            mean_loss += loss.item()

            if global_iteration % log_step == 0:
                mean_loss /= log_step
                progress_bar.set_postfix(dict(loss='{:.2}'.format(mean_loss)))
                mean_loss = 0

            if global_iteration % val_step == 0:
                # validate (using different thresholds)
                metrics = validate(val_dataloader, model, classes, thresholds=[0.3, 0.5, 0.8])

                # save best model
                if metrics['macroF1_thr=0.3'] + metrics['microF1_thr=0.3'] > best_f1:
                    print('Saving best model...')
                    checkpoint = {
                        'cfg': config,
                        'epoch': epoch,
                        'model': model.joint_processing_module.state_dict() if not config['text-model']['fine-tune'] and not config['image-model']['fine-tune'] else model.state_dict()}
                    latest = os.path.join(path, 'model_best_fold{}.pt'.format(val_fold))
                    torch.save(checkpoint, latest)
                    best_f1 = metrics['macroF1_thr=0.3'] + metrics['microF1_thr=0.3']

        print('Saving best model...')
        checkpoint = {
          'cfg': config,
          'epoch': epoch,
          'model': model.joint_processing_module.state_dict() if not config['text-model']['fine-tune'] and not config['image-model']['fine-tune'] else model.state_dict()}
        latest = os.path.join(path, 'model_best_fold{}.pt'.format(val_fold))
        torch.save(checkpoint, latest)
        best_f1 = metrics['macroF1_thr=0.3'] + metrics['microF1_thr=0.3']
        scheduler.step()

In [6]:
def validate(val_dataloader, model, classes_list, thresholds=[0.3, 0.5, 0.8]):
    model.eval()
    predictions = []
    metrics = {}
    progress_bar = tqdm.tqdm(thresholds)
    progress_bar.set_description('Validation')
    for thr in progress_bar:
        for it, (image, text, text_len, labels, ids) in enumerate(val_dataloader):
            if torch.cuda.is_available():
                image = image.cuda() if image is not None else None
                text = text.cuda()
                labels = labels.cuda()
            with torch.no_grad():
                pred_classes = model(image, text, text_len, inference_threshold=thr)

            for id, labels in zip(ids, pred_classes):    # loop over every element of the batch
                predictions.append({'id': id, 'labels': labels})

        macro_f1, micro_f1 = evaluate(predictions, val_dataloader.dataset.targets, classes_list)
        metrics['macroF1_thr={}'.format(thr)] = macro_f1
        metrics['microF1_thr={}'.format(thr)] = micro_f1

    model.train()
    return metrics

# Training parameters

In [7]:
def start_training(config, path, val_fold=0):
    num_epochs = 80
    workers = 2
    log_step = 10
    val_step = 200
    test_step = 100000000
    logger_name = './runs/test'

    if 'task' not in config['dataset']:
        config['dataset']['task'] = 3

    train_dataloader, val_dataloader, classes = load_data(config, path, workers, val_fold)

    # Construct the model
    model = MemeMultiLabelClassifier(config, labels=classes)
    if torch.cuda.is_available():
        model.cuda()

    # Construct the optimizer
    if not config['text-model']['fine-tune'] and not config['image-model']['fine-tune']:
        optimizer = torch.optim.Adam([p for n, p in model.named_parameters() if 'textual_module' not in n and 'visual_module' not in n], lr=config['training']['lr'])
    else:
        if config['dataset']['task'] == 3:
            optimizer = torch.optim.Adam([
                {'params': [p for n, p in model.named_parameters() if 'textual_module' not in n and 'visual_module' not in n]},
                {'params': model.textual_module.parameters(), 'lr': config['training']['pretrained-modules-lr']},
                {'params': model.visual_module.parameters(), 'lr': config['training']['pretrained-modules-lr']}]
                , lr=config['training']['lr'])

    # LR scheduler
    scheduler_name = config['training']['scheduler']
    if scheduler_name == 'steplr':
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, gamma=config['training']['gamma'], milestones=config['training']['milestones'])
    elif scheduler_name is None:
        scheduler = None
    else:
        raise ValueError('{} scheduler is not available'.format(scheduler_name))

    # Train the model
    train_model(config, path, model, optimizer, scheduler, train_dataloader, val_dataloader, classes, num_epochs, log_step, val_step, val_fold)


# Main function

In [8]:
def main():

    config = path + 'cfg/config_task3.yaml'
    cross_validation = False

    with open(config, 'r') as ymlfile:
        config = yaml.safe_load(ymlfile)

    if cross_validation:
        # read splits from file
        with open(path + 'data/folds.json', 'r') as f:
            folds = json.load(f)
            num_folds = len(folds)
        for fold in tqdm.trange(num_folds):
            start_training(config, path, val_fold=fold)
    else:
        # train using fold 0 as validation fold
        start_training(config, path, val_fold=0)

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_32-d86f8d99.pth" to /root/.cache/torch/hub/checkpoints/vit_b_32-d86f8d99.pth
100%|██████████| 337M/337M [00:06<00:00, 55.8MB/s]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.encoder.layer.10.attention.self.key.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.10.output.dense.bias', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.11.attention.self.key.bias', 'bert.encoder.layer.11.output.dense.weight', 'bert.encoder.layer.10.attention.self.query.bias', 'cls.predictions.transform.dense.weight', 'bert.encoder.layer.10.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.11.output.LayerNorm.bias', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.10.output.LayerNorm.weight', 'bert.encoder.layer.10.intermediate.dense.bias', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.11.attention.self.value.bias', 'bert.encode

Saving best model...


Train:   0%|          | 0/80 [02:42<?, ?it/s, loss=0.25]

Saving best model...


...

Train:  98%|█████████▊| 78/80 [19:46<00:25, 12.59s/it, loss=0.0018]

Saving best model...


Train:  99%|█████████▉| 79/80 [20:02<00:12, 12.23s/it, loss=0.0024]

Saving best model...


Train: 100%|██████████| 80/80 [20:04<00:00, 15.06s/it, loss=0.0024]


# Testing the trained model

In [9]:
%cd /content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection

/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection


In [29]:
!python inference.py --checkpoint model_best_fold0.pt --threshold 0.4 --validate

Namespace(threshold=0.4, checkpoint='model_best_fold0.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'bert.encoder.layer.11.intermediate.dense.bias', 'bert.encoder.layer.11.intermediate.dense.weight

In [ ]:
!python inference.py --checkpoint model_best_fold0.pt --threshold 0.5 --validate

Namespace(threshold=0.5, checkpoint='model_best_fold0.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.encoder.layer.11.intermediate.dense.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'bert.encoder.layer.10.attention.ou

In [ ]:
!python inference.py --checkpoint model_best_fold0.pt --threshold 0.8 --validate

Namespace(threshold=0.8, checkpoint='model_best_fold0.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.encoder.layer.11.attention.self.query.bias', 'bert.encoder.layer.10.output.dense.weight', 'bert.encoder.layer.11.output.dense.weigh

In [ ]:
!python inference.py --checkpoint model_best_fold0.pt --threshold 0.9 --validate

Namespace(threshold=0.9, checkpoint='model_best_fold0.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.encoder.layer.11.attention.self.key.bias', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.10.intermediate.dense.wei

## In case there is other saved models

In [ ]:
!python inference.py --checkpoint model_best_fold1.pt --validate

Namespace(threshold=0.3, checkpoint='model_best_fold1.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
Traceback (most recent call last):
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 178, in <module>
    main(opt)
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 30, in main
    checkpoint = torch.load(opt.checkpoint, map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'model_best_fold1.pt'


In [ ]:
!python inference.py --checkpoint model_best_fold2.pt --validate

Namespace(threshold=0.3, checkpoint='model_best_fold2.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
Traceback (most recent call last):
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 178, in <module>
    main(opt)
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 30, in main
    checkpoint = torch.load(opt.checkpoint, map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'model_best_fold2.pt'


In [ ]:
!python inference.py --checkpoint model_best_fold3.pt --validate

Namespace(threshold=0.3, checkpoint='model_best_fold3.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
Traceback (most recent call last):
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 178, in <module>
    main(opt)
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 30, in main
    checkpoint = torch.load(opt.checkpoint, map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'model_best_fold3.pt'


In [ ]:
!python inference.py --checkpoint model_best_fold4.pt --validate

Namespace(threshold=0.3, checkpoint='model_best_fold4.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
Traceback (most recent call last):
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 178, in <module>
    main(opt)
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 30, in main
    checkpoint = torch.load(opt.checkpoint, map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'model_best_fold4.pt'


In [ ]:
!python inference.py --checkpoint model_best_fold5.pt --validate

Namespace(threshold=0.3, checkpoint='model_best_fold5.pt', validate=True, test=False, val_fold=0, ensemble=False, cross_validation=False)
Traceback (most recent call last):
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 178, in <module>
    main(opt)
  File "/content/drive/MyDrive/Telecom/Airbus/Persusasion/MemePersuasionDetection/inference.py", line 30, in main
    checkpoint = torch.load(opt.checkpoint, map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'model_best_fold5.pt'
